In [1]:
pip install transformers

     |████████████████████████████████| 2.6 MB 6.8 MB/s 
     |████████████████████████████████| 636 kB 55.8 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
     |████████████████████████████████| 3.3 MB 62.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")

model = AutoModelForSequenceClassification.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")

In [3]:
import pandas as pd
xls = pd.ExcelFile('Негатив_пример.xlsx')
xls_toxic = pd.read_excel('Негатив_пример.xlsx', index_col=None, header=0)  

In [9]:
#for i in range(len(xls_toxic['Text'])):
pt_batches_full = tokenizer(
  list(xls_toxic['Text']),
  padding=True,    truncation=True,    max_length=512,    return_tensors="pt" )

In [14]:
pt_batches_full[10]

Encoding(num_tokens=97, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [16]:
from torch import nn
pt_predictions_full = []
pt_outputs = model(**pt_batches_full)
pt_prediction_full = nn.functional.softmax(pt_outputs.logits, dim=-1)
pt_predictions_full.append(pt_prediction_full)

In [31]:
toxicity = [float(pt_prediction_full[i][1]) for i in range(len(pt_prediction_full))]

In [ ]:
toxicity

In [18]:
version_full = pd.DataFrame()

In [19]:
version_full['Text'] = xls_toxic['Text']

In [33]:
version_full['Toxicity'] = toxicity

In [34]:
version_full

,Text,Toxicity
0,ужасная поддержка!,0.044271
1,ужасная поддержка,0.003017
2,"где мой деньги паскуды, сколько можно херней з...",0.029713
3,"завтра же нахер все позакрою тут, ебучее говно",0.978019
4,У вас приложение говно полное,0.032108
...,...,...
105,"скажите, где и кому сообщать по поводу работы ...",0.002664
106,что за хуйня опять у вас там творится,0.993832
107,потому по щётку такая хуйня,0.995443
108,"что значит ""зафиксировать обращение""? я хочу в...",0.002268


In [36]:
version_full.to_excel('негатив_вывод_ver_2(полные комменты).xlsx')

In [ ]:
-\\\-

In [109]:
xls_toxic['Text'][3].split(' ')

['завтра', 'же', 'нахер', 'все', 'позакрою', 'тут,', 'ебучее', 'говно']

In [ ]:
pt_batches = []
for i in range(len(xls_toxic['Text'])):
  #print(xls_toxic['Text'][i].split(' '))
  pt_batch = tokenizer(
    list(xls_toxic['Text'][i].split(' ')),
    padding=True,    truncation=True,    max_length=512,    return_tensors="pt" )
  pt_batches.append(pt_batch)

In [133]:
from torch import nn
pt_predictions = []
for pt_batch in pt_batches:
  pt_output = model(**pt_batch)
  pt_prediction = nn.functional.softmax(pt_output.logits, dim=-1)
  pt_predictions.append(pt_prediction)

In [137]:
pt_predictions[5]

tensor([[0.0545, 0.9455]], grad_fn=<SoftmaxBackward>)

In [166]:
xls_toxic['Text'][3], pt_predictions[3]

('завтра же нахер все позакрою тут, ебучее говно', tensor([[0.9566, 0.0434],
         [0.5721, 0.4279],
         [0.0158, 0.9842],
         [0.9861, 0.0139],
         [0.6752, 0.3248],
         [0.9926, 0.0074],
         [0.0190, 0.9810],
         [0.0545, 0.9455]], grad_fn=<SoftmaxBackward>))

In [185]:
import numpy as np
bad_words_full = []
procents = []
for i in range(len(pt_predictions)):
  bad_words = []
  words = xls_toxic['Text'][i].split(' ')
  #print(float(pt_predictions[i][0][1])) 
  for j in range(len(pt_predictions[i])):
    if float(pt_predictions[i][j][1]) > 0.83:
      #print(type(words[j]))
      bad_words.append((words[j], np.round(float(pt_predictions[i][j][1]), 4)))
      #bad_words.append(float(pt_predictions[i][j][1]))
  procent = len(bad_words)/len(words)
  procents.append(procent)
  bad_words_full.append(bad_words)
print(bad_words_full)

[[], [], [('паскуды,', 0.9803), ('херней', 0.9247)], [('нахер', 0.9842), ('ебучее', 0.981), ('говно', 0.9455)], [('говно', 0.9455)], [('Говно', 0.9455)], [('хрень', 0.8584)], [('хрена', 0.9427)], [], [('издеваетесь', 0.9586)], [], [], [('издеваетесь?', 0.9623)], [], [], [('Пидары', 0.9952), ('ебаные', 0.9942), ('!!!', 0.8409)], [], [], [], [('Уроды', 0.9934), ('ебаные', 0.9942)], [], [], [('они', 0.8817), ('херня', 0.9859)], [('издевается?', 0.9308)], [], [], [('херня', 0.9859)], [], [('херня', 0.9859)], [('блять', 0.9783), ('хуйня', 0.9889)], [('блять', 0.9783)], [], [], [('добрый', 0.9257)], [], [], [('издеваетесь', 0.9586)], [('Дикий', 0.9748)], [('издеваетесь', 0.9586)], [('издеваетесь?', 0.9623)], [], [('Помоему', 0.9192), ('издеваетесь', 0.9586)], [], [], [('блядь', 0.9811), ('капейки', 0.8845)], [('ебанутая', 0.9937)], [('издеваетесь', 0.9586)], [('Издеваетесь?', 0.9623)], [], [], [], [], [], [('своим', 0.8412), ('тупым', 0.9918), ('пялиться', 0.9895)], [('издеваетесь?', 0.9623)

In [186]:
xls_toxic['Bad_words'] = bad_words_full
xls_toxic['Percentage of bad'] = np.round(procents, 4)

In [187]:
xls_toxic

,Text,Bad_words,Percentage of bad
0,ужасная поддержка!,[],0.000
1,ужасная поддержка,[],0.000
2,"где мой деньги паскуды, сколько можно херней з...","[(паскуды,, 0.9803), (херней, 0.9247)]",0.250
3,"завтра же нахер все позакрою тут, ебучее говно","[(нахер, 0.9842), (ебучее, 0.981), (говно, 0.9...",0.375
4,У вас приложение говно полное,"[(говно, 0.9455)]",0.200
...,...,...,...
105,"скажите, где и кому сообщать по поводу работы ...","[(говорите,, 0.8582)]",0.050
106,что за хуйня опять у вас там творится,"[(хуйня, 0.9889)]",0.125
107,потому по щётку такая хуйня,"[(хуйня, 0.9889)]",0.200
108,"что значит ""зафиксировать обращение""? я хочу в...",[],0.000


In [189]:
xls_toxic.to_excel('негатив_вывод.xlsx')

In [ ]:
BAD = []
for i in range(len(pt_predictions)):
  bad_words = []
  for j in range(len(xls_toxic['Text'][i].split(' '))):
    for word in pt_predictions[i]:
      if word[1] > 0.8:
        bad_words.append([xls_toxic['Text'][i].split(' ')[j], word[1]])
  BAD.append(bad_words)
print(xls_toxic['Text'][i], BAD[i])